In [1]:
import sys

sys.path.insert(0, r"C:\Users\lukas\Documents\projects\testing\invert-old\invert")
import invert as invert1
Solver1 = invert1.Solver

# remove invert from sys.modules
if 'invert' in sys.modules:
    del sys.modules['invert']

sys.path.insert(0, r"C:\Users\lukas\Documents\projects\testing\invert-new\invert")
import invert as invert2
Solver2 = invert2.Solver

print(invert1.__file__ == invert2.__file__)


False


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# import sys; sys.path.insert(0, '../') 
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
from scipy.spatial.distance import cdist
import mne

from invert.forward import get_info, create_forward_model
from invert.util import pos_from_forward
from invert.evaluate import eval_auc, eval_mean_localization_error, eval_mean_localization_error_old

pp = dict(surface='white', hemi='both', verbose=0)

In [3]:
sampling = "ico4"
info = get_info(kind='biosemi128')
fwd = create_forward_model(info=info, sampling=sampling)
pos = pos_from_forward(fwd)
leadfield = fwd["sol"]["data"]
n_chans, n_dipoles = leadfield.shape

source_model = fwd['src']
vertices = [source_model[0]['vertno'], source_model[1]['vertno']]
adjacency = mne.spatial_src_adjacency(fwd["src"], verbose=0)
distance_matrix = cdist(pos, pos)
fwd

c:\Users\lukas\miniconda3\envs\flap2\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Good channels,128 EEG
Bad channels,None
Source space,Surface with 5124 vertices
Source orientation,Fixed


# Simulate

In [4]:
from invert.simulate import generator
sim_params = dict(
    use_cov=False,
    return_mask=False,
    batch_repetitions=1,
    batch_size=1,
    n_sources=3,
    n_orders=(3, 5),
    snr_range=(1, 1),
    amplitude_range=(1, 1),
    n_timecourses=200,
    n_timepoints=50,
    scale_data=False,
    add_forward_error=False,
    forward_error=0.,
    inter_source_correlation=0.5,
    iid_noise=True, 
    return_info=True,
    diffusion_parameter=0.1,
    random_seed=None)

gen = generator(fwd, **sim_params)

In [5]:
x, y, sim_info = gen.__next__()

tmin = 0
tstep = 1/info["sfreq"]
subject = "fsaverage"
evoked = mne.EvokedArray(x[0].T, info, tmin=tmin)
stc = mne.SourceEstimate(y[0].T, vertices, tmin=tmin, tstep=tstep, 
                        subject=subject, verbose=0)

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, "Ground Truth", 'title',
               font_size=14)
evoked.plot_joint(title="Ground Truth")

evoked_clean = mne.EvokedArray(leadfield @ y[0].T, info, tmin=tmin)
evoked_clean.plot_joint(title="Ground Truth Noiseless")

sim_info

No projector specified for this dataset. Please consider the method self.add_proj.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
No projector specified for this dataset. Please consider the method self.add_proj.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


,n_sources,amplitudes,snr,inter_source_correlations,n_orders,diffusion_parameter,n_timepoints,n_timecourses,iid_noise
0,3,"[1.0, 1.0, 1.0]",1.0,0.5,"[3, 5]",0.1,50,200,True


# Solve

## FLAP

In [56]:
from invert import Solver

solver = Solver1("AP")
solver.make_inverse_operator(fwd, evoked, n_orders=0, refine_solution=True, n=n_solver, k=n_solver, diffusion_parameter=0.1)
# solver.make_inverse_operator(fwd, evoked, n_orders=3, refine_solution=True, n=sim_params["n_sources"], d=sim_params["n_sources"], diffusion_parameter=0.05)
stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")
print( len(np.where((stc_.data**2).sum(axis=1) != 0)[0]), " sources")

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\lukas\AppData\Local\Temp\ipykernel_7208\2114689780.py:8: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
[2473 4641 4750]
[3334]
Alternating Projections mle =  44.83179670480428  mm
1  sources


In [57]:
from invert import Solver

solver = Solver2("AP")
solver.make_inverse_operator(fwd, evoked, n_orders=0, refine_solution=True, n=n_solver, k=n_solver, diffusion_parameter=0.1)
# solver.make_inverse_operator(fwd, evoked, n_orders=3, refine_solution=True, n=sim_params["n_sources"], d=sim_params["n_sources"], diffusion_parameter=0.05)
stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")
print( len(np.where((stc_.data**2).sum(axis=1) != 0)[0]), " sources")

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\lukas\AppData\Local\Temp\ipykernel_7208\1474159169.py:8: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
[2473 4641 4750]
[3334]
Alternating Projections mle =  44.83179670480428  mm
1  sources


In [58]:
from invert import Solver

solver = Solver1("RAP-MUSIC")
solver.make_inverse_operator(fwd, evoked, n_orders=0, refine_solution=False, n=n_solver, k=n_solver, diffusion_parameter=0.1, stop_crit=0)
# solver.make_inverse_operator(fwd, evoked, n_orders=3, refine_solution=True, n=sim_params["n_sources"], d=sim_params["n_sources"], diffusion_parameter=0.05)
stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")
print( len(np.where((stc_.data**2).sum(axis=1) != 0)[0]), " sources")

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\lukas\AppData\Local\Temp\ipykernel_7208\70374327.py:8: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
[2473 4641 4750]
[3038]
RAP-MUSIC mle =  49.12864947282924  mm
1  sources


In [59]:
from invert import Solver

solver = Solver2("RAP-MUSIC")
solver.make_inverse_operator(fwd, evoked, n_orders=0, refine_solution=False, n=n_solver, k=n_solver, diffusion_parameter=0.1, stop_crit=0)
# solver.make_inverse_operator(fwd, evoked, n_orders=3, refine_solution=True, n=sim_params["n_sources"], d=sim_params["n_sources"], diffusion_parameter=0.05)
stc_ = solver.apply_inverse_operator(evoked)
stc_.data /= abs(stc_.data).max()
brain = stc_.plot(**pp)
brain.add_text(0.1, 0.9, solver.name, 'title',
               font_size=14)

evoked_ = mne.EvokedArray(fwd["sol"]["data"] @ stc_.data, info).set_eeg_reference("average", projection=True)
evoked_.plot_joint()

# print(solver.name, " r = ", pearsonr(abs(stc.data).mean(axis=-1), abs(stc_.data).mean(axis=-1))[0])

mle = eval_mean_localization_error(stc.data, stc_.data, adjacency.toarray(), adjacency.toarray(), distance_matrix)
print(solver.name, "mle = ", mle, " mm")
print( len(np.where((stc_.data**2).sum(axis=1) != 0)[0]), " sources")

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\lukas\AppData\Local\Temp\ipykernel_7208\3895963447.py:8: RuntimeWarning: All data were zero
  brain = stc_.plot(**pp)


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
[2473 4641 4750]
[3038]
RAP-MUSIC mle =  49.12864947282924  mm
1  sources
